<a href="https://colab.research.google.com/github/piziomo/Data-Science/blob/main/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Regressions

In this lab, inspired by @igual_regression_2017, we will introduce simple linear regression in Python to try to answer the question: _Has there been a decrease in the amount of ice in the last years?_ To do so, we are going to use `Scikit Learn`, a machine learning library, and the [Sea Ice Index Daily and Monthly Image Viewer](https://nsidc.org/data/seaice_index) dataset from the National Snow and Ice Data Center.



## Key concepts' refresher

Let's refresh some theoretical concepts to understand what we are going to do.

### Simple and Multiple Linear Regression

In the **linear model** the response $\textbf{y}$ depends linearly from the covariates $\textbf{x}_i$.

In the **simple** linear regression, with a single variable, we described the relationship between the predictor and the response with a straight line. The general linear model:
$$ \textbf{y}  =  a_0+ a_1 \textbf{x}_1 $$

The parameter $a_0$ is called the *constant* term or the *intercept*.

In the case of **multiple** linear regression we extend this idea by fitting a m-dimensional hyperplane to our m predictors.

$$ \textbf{y}  =  a_1 \textbf{x}_1  + \dots + a_m \textbf{x}_{m} $$

The $a_i$ are termed the *parameters* of the model or the coefficients.

### Ordinary Least Squares

Ordinary Least Squares (OLS) is the simplest and most common **estimator** in which the coefficients $a$'s
of the simple linear regression: $\textbf{y} = a_0+a_1 \textbf{x}$,
are chosen to minimize the **square of the distance between the predicted values and the actual values**.

$$ ||a_0 + a_1 \textbf{x} -  \textbf{y} ||^2_2 = \sum_{j=1}^n (a_0+a_1 x_{j} -  y_j )^2,$$

This expression is often called **sum of squared errors of prediction (SSE)**.

## Case study: Climate Change and Sea Ice Extent

Remember, we are trying to answer the following research question: _Has there been a decrease in the amount of ice in the last years?_

### Data assessment

First, let's load the `SeaIce.txt` dataset that is already in the `data` folder[^seaice]. It is a text file, but unlike `csv` files, where columns are separated by commas (`,`), it is a `Tab` separated file, where each `Tab` delimites the following columns:

- `Year`:	4-digit year
- `mo`:	1- or 2-digit month
- `data_type`:	Input data set (Goddard/NRTSI-G)
- `region`:	Hemisphere that this data covers (N: Northern; S: Southern)
- `extent`:	Sea ice extent in millions of square km
- `area`:	Sea ice area in millions of square km

Once we upload the data, we can create a `DataFrame` using Pandas using the well known `read_csv()` function, but in this case, because columns are not separated by commas as expected, but Tabs, we will need to use the `delim_whitespace=True` argument.

[^seaice]: The original dataset was downloaded from <https://nsidc.org/data/g02135/versions/3>, which provides useful metadata information, as well as API services.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
ice = pd.read_csv('data/raw/SeaIce.txt', delim_whitespace=True)

ice.info()

In [ ]:
ice.head()

And we can get some summary statistics from the numerical attributes:

In [ ]:
ice.describe()

::: callout-warning

Did we receive a negative mean for `extent` and `area`? What this could possibly mean? Probably, inspecting those attributes visually could give us a clue.

:::

### Data visualisation to explore data

We will use Seaborn's `lmplot()` function to explore linear relationship of different forms, e.g. relationship between the `month` of the `year` (variable) and `extent` (responses):

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Visualize the data
plt.scatter(ice.mo, ice.extent, color = 'red')
plt.xlabel('Year')
plt.ylabel('Extent')

::: callout-warning

We detect some outlier or missing data. This might have to do with those negative mean values that we detected previously.

:::

We can use numpy's function [`np.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) to find the unique elements of an array.

In [ ]:
print ('Different values in data_type field:', np.unique(ice.data_type.values))   # there is a -9999 value!

Let's see what type of data we have, other than the ones printed above

In [ ]:
ice[(ice.data_type != 'Goddard') & (ice.data_type != 'NRTSI-G')]

### Data cleaning

We checked all the values and notice `-9999` values in data_type field which should contain `Goddard` or `NRTSI-G` (some type of input dataset).

In this case, we will clean them by creating a _copy_ of the original dataframe that does not include these instances.

::: callout-important

### Dataframe copies vs instances

Unless we do not explicitly create a copy of a dataframe, when subsetting a dataframes we are actually creating instances. Whereas copies are totally independent objects from the original one, instances are reduced "views" from the original, meaning that if we change a value on the instance, we are also changing the value on the original data frame, which may not be what we wanted to do.

An explanation of how (and why) do we need to create copies can be found here: <https://www.statology.org/pandas-copy-dataframe/>

:::

In [ ]:
# We can easily clean the data now:
ice2 = ice[ice.data_type != '-9999'].copy()

print ('shape:', ice2.shape)

In [ ]:
# And repeat the plot, without the outliers
plt.scatter(ice2.mo, ice2.extent, color = 'red')
plt.xlabel('Month')
plt.ylabel('Extent')

In [ ]:
#| column: page-right

#here is the same plot but using the seaborn library. A transition to the Seaborn plot we have in the next cell.

#sns.relplot(ice2, x = "mo", y = "extent", aspect = 2)

## Regression model fit

Now that we have a clean dataset, we can use Seaborn's [`lmplot()`](https://seaborn.pydata.org/generated/seaborn.lmplot.html) function comparing month vs extent.

The `lmplot()` function from the Seaborn module is intended for exploring linear relationships of different forms in multidimensional datesets. Input data must be in a Pandas DataFrame. To plot them, we provide the predictor (`mo`) and response (`extent`) variable names along with the dataset (`ice2`).

In [ ]:
#| column: page-right

sns.lmplot(ice2, x = "mo", y = "extent", aspect=2);

# Uncomment below to save the resulting plot.
#plt.savefig("figs/CleanedByMonth.png", dpi = 300, bbox_inches = 'tight')

Above you can see ice extent data by month.
You can see a monthly fluctuation of the sea ice extent, as would be expected for the different seasons of the year. In order to run regression, and avoid this fluctuation we can normalize data. This will let us see the evolution of the extent over the years.

### Normalization


In [ ]:
# Compute the mean for each month.
month_means = ice2.groupby('mo').extent.mean()

# Compute the variance for each month.
month_variances = ice2.groupby('mo').extent.var()

# Show the values:
print('Means:', month_means)
print('\n') # Add a new line between the two prints, so they are easily distinguishible.
print ('Variances:',month_variances)

To capture variation per month, we can compute mean for the i-th interval of time (using 1979-2014) and subtract it from the set of extent values for that month . This can be converted to a relative pecentage difference by dividing it by the total avareage (1979-2014) and multiplying by 100.

In [ ]:
# Let's create a new column to hold these normalised values.
ice2['extent_norm'] = np.nan

In [ ]:
# run the following to check what the data types look like:
ice2.dtypes
ice2.head()

In [ ]:
# Data normalization loop. Note that we are saving the new computed values into the
for i in range(12):
    ice2.extent_norm[ice2.mo == i+1] = 100*(ice2.extent[ice2.mo == i+1] - month_means[i+1])/month_means.mean()
    #print ("ice2.extent[ice2.mo == i+1]", 100*(ice2.extent[ice2.mo == i+1] - month_means[i+1])/month_means.mean())
    #print(month_means.mean())

In [ ]:
# let's check if all is in place.
ice2.head()

In [ ]:
sns.lmplot(ice2 , x = "mo", y = "extent_norm", height = 5.2, aspect = 2);
#plt.savefig("figs/IceExtentNormalizedByMonth.png", dpi = 300, bbox_inches = 'tight')

In [ ]:
print ('mean:', ice2.extent.mean())
print ('var:', ice2.extent.var())

Let us consider the entire year

In [ ]:
sns.lmplot(ice2, x = "year", y = "extent", height = 5.2, aspect = 2);
#plt.savefig("figs/IceExtentAllMonthsByYearlmplot.png", dpi = 300, bbox_inches = 'tight')

::: callout-caution
## Important Do-it-Yourself Moment

A question here! Would you like to use the variable `extent` or `extent_norm` here. What would this decision change? Reflect on this, or better, try out the above with different versions of the data. Note observations.

:::

### Pearson's correlation

Let's calculate Pearson's correlation coefficient and the p-value for testing non-correlation.

In [ ]:
import scipy.stats
scipy.stats.pearsonr(ice2.year.values, ice2.extent.values)

### Simple OLS

We can also compute the trend as a simple linear regression (OLS) and quantitatively evaluate it.

For that we use using **Scikit-learn**, library that provides a variety of both supervised and unsupervised machine learning techniques.
Scikit-learn provides an object-oriented interface centered around the concept of an Estimator.
The <code>Estimator.fit</code> method sets the state of the estimator based on the training data. Usually, the data is comprised of a two-dimensional numpy array $X$ of shape <code>(n_samples, n_predictors)</code> that holds the so-called feature matrix and a one-dimensional numpy array $\textbf{y}$ that holds the responses. Some estimators allow the user to control the fitting behavior.
For example, the <code>sklearn.linear_model.LinearRegression</code> estimator allows the user to specify whether or not to fit an intercept term. This is done by setting the corresponding constructor arguments of the estimator object.
During the fitting process, the state of the estimator is stored in instance attributes that have a trailing underscore (`_`). For example, the coefficients of a LinearRegression estimator are stored in the attribute `coef_`.

Estimators that can generate predictions provide a `Estimator.predict` method.
In the case of regression, `Estimator.predict` will return the predicted regression values, $\hat{\textbf{y}}$.

In [ ]:
from sklearn.linear_model import LinearRegression

est = LinearRegression(fit_intercept = True)

x = ice2[['year']]
y = ice2[['extent']]

est.fit(x, y)

print("Coefficients:", est.coef_)
print ("Intercept:", est.intercept_)

We can evaluate the model fitting by computing the mean squared error ($MSE$) and the coefficient of determination ($R^2$) of the model.
The coefficient $R^2$ is defined as $(1 - \textbf{u}/\textbf{v})$, where $\textbf{u}$ is the residual sum of squares $\sum (\textbf{y} - \hat{\textbf{y}})^2$ and $\textbf{v}$ is the regression sum of squares $\sum (\textbf{y} - \bar{\textbf{y}})^2$, where $\bar{\textbf{y}}$ is the mean.
The best possible score for $R^2$ is 1.0: lower values are worse.
These measures can provide a quantitative answer to the question we are facing: Is there a negative trend in the evolution of sea ice extent over recent years?

In [ ]:
from sklearn import metrics

# Analysis for all months together.
x = ice2[['year']]
y = ice2[['extent']]
model = LinearRegression()
model.fit(x, y)
y_hat = model.predict(x)
plt.plot(x, y,'o', alpha = 0.5)
plt.plot(x, y_hat, 'r', alpha = 0.5)
plt.xlabel('year')
plt.ylabel('extent (All months)')
print ("MSE:", metrics.mean_squared_error(y_hat, y))
print ("R^2:", metrics.r2_score(y_hat, y))
print ("var:", y.var())
#plt.savefig("figs/IceExtentLinearRegressionAllMonthsByYearPrediction.png", dpi = 300, bbox_inches = 'tight')

We can conclude that the data show a long-term negative trend in recent years.

In [ ]:
# Analysis for a particular month.
#For January
jan = ice2[ice2.mo == 1];

x = jan[['year']]
y = jan[['extent']]

model = LinearRegression()
model.fit(x, y)

y_hat = model.predict(x)

plt.figure()
plt.plot(x, y,'-o', alpha = 0.5)
plt.plot(x, y_hat, 'r', alpha = 0.5)
plt.xlabel('year')
plt.ylabel('extent (January)')

print ("MSE:", metrics.mean_squared_error(y_hat, y))
print ("R^2:", metrics.r2_score(y_hat, y))

We can also estimate the extent value for 2025. For that we use the function predict of the model.

In [ ]:
X = np.array(2025)
y_hat = model.predict(X.reshape(-1, 1))
j = 1 # January
# Original value (before normalization)
y_hat = (y_hat*month_means.mean()/100) + month_means[j]
print ("Prediction of extent for January 2025 (in millions of square km):", y_hat)

## More information

You can find more information about using `scikit` for Linear regression here: <https://www.tutorialspoint.com/scikit_learn/scikit_learn_linear_regression.htm>